In [1]:
from collections import deque
import numpy as np
import random

DIRECTIONS = ((1, 0), (0, -1), (-1, 0), (0, 1))  # Down, Left, Up, Right
ARROWS = ("↓", "↲", "↳", "←", "↰", "↲", "↑", "↱", "↰", "→", "↳", "↱") # "↶" for U

def coord_plus_direction(coord: tuple, direction: int):
    return (coord[0] + DIRECTIONS[direction % 4][0], coord[1] + DIRECTIONS[direction % 4][1])

In [55]:
@staticmethod
class Cell:
    """Represents a single cell on the map."""

    def __init__(self, coords, lane, entry=False, exit=False, accessible=True, heading_direction=2, car_list=None, toggle_group=None):
        self.coords = coords # (Y, X) coordinates
        self.entry = entry # False for no entry , True for entry position
        self.exit = exit # False for not exit, True for exit position
        self.accessible = accessible  # False for inaccessible, True for accessible
        self.heading_direction = heading_direction # 0 for down, 1 for left, 2 for up, 3 for right
        self.lane = lane # 1 for left border, 0 for middle lanes, 2 for right border, 3 for solo lanes
        self.car_list = deque(car_list or [], maxlen=2) # deque of cars in Cell
        self.toggle_group = toggle_group  # Group for traffic lights

    def __str__(self):
        return str(self.car_list[0]) if self.car_list else 'X' if not self.accessible else 'S' if self.entry else 'D' if self.exit else str(self.lane)
    
    def change_car_list(self, new_car_list):
        return Cell(coords=self.coords, lane=self.lane, entry=self.entry, exit=self.exit, accessible=self.accessible, heading_direction=self.heading_direction, car_list=new_car_list, toggle_group=self.toggle_group)

    def toggle_cell_accessibility(self):
        toggled_accessibility = not self.accessible
        return Cell(coords=self.coords, lane=self.lane, entry=self.entry, exit=self.exit, accessible=toggled_accessibility, heading_direction=self.heading_direction, car_list=self.car_list, toggle_group=self.toggle_group)

class Road:
    """Represents the map and runs the simulation."""
    def __init__(self, grid):
        self.grid = grid
        self.start_coords = [(y, x) for y in range(grid.shape[0]) for x in range(grid.shape[1]) if grid[y, x].entry] 
        self.toggle_group_dict = self.find_toggle_group_dict()
    
    def find_toggle_group_dict(self):
        """Creates a dictionary mapping toggle_group values to lists of corresponding cell coordinates."""
        toggle_group_coords = {}
        for row in self.grid:
            for cell in row:
                toggle_group_coords.setdefault(cell.toggle_group, []).append(cell.coords)
        return toggle_group_coords
    
    def first_car_moves(self, cell_a: Cell):
        if cell_a.car_list:
            first_car = cell_a.car_list[0]
            cell_b = self.grid[first_car.next_move[0]]
            if cell_b is not None and cell_b.accessible:
                    # Update car lists: original cell removes first car, next move cell appends the updated car object to car list
                    new_car_list_a = cell_a.car_list.copy()
                    new_car_list_a.remove(first_car)

                    new_car_list_b = cell_b.car_list.copy()
                    new_car_list_b.append(Car(cell_b, first_car.id, self))

                    # Updating grid
                    updated_grid = self.grid.copy()
                    updated_grid[cell_a.coords] = Cell(coords=cell_a.coords, lane=cell_a.lane, entry=cell_a.entry, exit=cell_a.exit, accessible=cell_a.accessible, heading_direction=cell_a.heading_direction, car_list=new_car_list_a, toggle_group=cell_a.toggle_group)
                    updated_grid[cell_b.coords] = Cell(coords=cell_b.coords, lane=cell_b.lane, entry=cell_b.entry, exit=cell_b.exit, accessible=cell_b.accessible, heading_direction=cell_b.heading_direction, car_list=new_car_list_b, toggle_group=cell_b.toggle_group)
                    self.grid = updated_grid
                    return True
        return False

    def toggle_traffic_lights(self, group: int):
        """Switches the state of traffic light cells every 4 timesteps."""
        toggle_group_members = self.toggle_group_dict[group] # Finds traffic light group's coords
        updated_grid = self.grid.copy()
        for coord in toggle_group_members: # Every Cell in group gets their accessibility toggled
            updated_grid[coord] = self.grid[coord].toggle_cell_accessibility()
        self.grid = updated_grid

    def step_grid(self, steps=10):
        """Runs the parallel simulation."""
        print("      start \n", self)
        for i in range(steps):
            for row in self.grid:
                for cell in row:
                    self.first_car_moves(cell)
            print(f"   timestep {i+1} \n", self)
            self.toggle_traffic_lights(1)
            

    def __str__(self):
        """String representation for visualization."""
        return str(np.vectorize(str)(self.grid))
    
    def grid_add_cars_at(self, car_coords: list):
        for coord in car_coords:
            if 0 <= coord[0] < self.grid.shape[0] and 0 <= coord[1] < self.grid.shape[1]: # only valid coords should get Car objects
                car = Car(cell=self.grid[coord], car_id=car_coords.index(coord), road=self) # create car
                updated_car_list = self.grid[coord].car_list.copy() # copy car list
                updated_car_list.append(car) # update car list to have car
                self.grid[coord] = self.grid[coord].change_car_list(new_car_list=updated_car_list) # add modified Cell to grid
    
@staticmethod
class Car:
    """Represents a car moving through the grid."""
    def __init__(self, cell: Cell, car_id: int, road: Road):
        self.cell = cell
        self.next_move = self.choose_next_move(road)
        self.id = car_id

    def choose_next_move(self, road: Road):
        """Find next coord for Cell"""
        if self.cell.exit:  # If at exit location, pick a random start location
            return (random.choice(road.start_coords), "✓")
        forward = road.grid[(coord_plus_direction(self.cell.coords, self.cell.heading_direction))]
        if forward is not None and (forward.accessible or (not forward.accessible and forward.toggle_group)):
            possible_moves = [(forward.coords, ARROWS[self.cell.heading_direction*3])]
            if self.cell.lane in {0, 1}:  # Rightward lane switch
                right = road.grid[(coord_plus_direction(forward.coords, self.cell.heading_direction + 1))]
                if right is not None and (right.accessible or (not right.accessible and right.toggle_group)):
                        possible_moves.append((right.coords, ARROWS[self.cell.heading_direction*3 + 1]))
            if self.cell.lane in {0, 2}:  # Leftward lane switch
                left = road.grid[(coord_plus_direction(forward.coords, self.cell.heading_direction - 1))]
                if left is not None and (left.accessible or (not left.accessible and left.toggle_group)):
                        possible_moves.append((left.coords, ARROWS[self.cell.heading_direction*3 + 2]))
            return random.choice(possible_moves)  

    def __str__(self): 
        return str(self.next_move[1])

In [49]:
def create_road(dims: tuple): #                                    (7 x 3)
        grid = np.empty(dims, dtype=object) #                        D D D
        for y in range(dims[0]): #                                   1 0 2
            exit = True if y == 0 else False #                       1 0 2
            entry = True if y == dims[0] - 1 else False #            S S S

            for x in range(dims[1]):
                lane = 1 if x == 0 else (2 if x == dims[1] - 1 else 0)
                if exit: # Setting exit cells to toggle group 1
                    grid[y, x] = Cell(coords=(y, x), lane=lane, entry=entry, exit=exit, toggle_group=1)
                else:
                    grid[y, x] = Cell(coords=(y, x), lane=lane, entry=entry, exit=exit)
        return Road(grid)

In [57]:
# --- Map Initialization ---
road: Road = create_road((7, 3))
#road.grid_add_cars_at([(5, 1), (6, 2), (4, 0), (4, 1)])
road.grid_add_cars_at([(5, 1), (5, 1), (5, 1), (4, 1)])


#for car in road.grid[(5, 1)].car_list:
#    print(car)

# --- Run Simulation ---
road.step_grid(steps=15)

      start 
 [['D' 'D' 'D']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '↱' '2']
 ['1' '↑' '2']
 ['S' 'S' 'S']]
   timestep 1 
 [['D' 'D' 'D']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '↑']
 ['1' '↰' '2']
 ['1' '↑' '2']
 ['S' 'S' 'S']]
   timestep 2 
 [['X' 'X' 'X']
 ['1' '0' '2']
 ['1' '0' '↰']
 ['↱' '0' '2']
 ['1' '↑' '2']
 ['1' '0' '2']
 ['S' 'S' 'S']]
   timestep 3 
 [['D' 'D' 'D']
 ['1' '↱' '2']
 ['1' '↰' '2']
 ['1' '↱' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['S' 'S' 'S']]
   timestep 4 
 [['X' 'X' 'X']
 ['↱' '↱' '2']
 ['1' '0' '↑']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['S' 'S' 'S']]
   timestep 5 
 [['D' '✓' '✓']
 ['1' '0' '↑']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['S' 'S' 'S']]
   timestep 6 
 [['X' 'X' 'X']
 ['1' '0' '↑']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['S' 'S' '↰']]
   timestep 7 
 [['D' 'D' '✓']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '0' '2']
 ['1' '↱' '2']
 ['S' 'S' '↰']]
   timestep 8 
 [['X' 'X' 

In [ ]:
car_list = np.array([None, None])

if car_list.any():
    print("we are not empty!!")
car_list[0] = 42
car_list[1] = 58
if car_list.any():
    print("we are not empty")

if car_list.all():
    print("you cannot go here")
else:
    if car_list[0] is None:
        print("you were put in slot one")
    else:
        print("you were put in slot two")


car_list[0] = car_list[1]  # Shift second item to first
car_list[1] = None

if car_list.all():
    print("you cannot go here")
else:
    if car_list[0] is None:
        print("you were put in slot one")
    else:
        print("you were put in slot two")

we are not empty
you cannot go here
you were put in slot two


In [47]:
car_list is np.array((4, 6))

if car_list:
    star_list = car_list
else:
    star_list = np.empty((2, 1), int)

print(star_list)

[[4]
 [6]]
